# Итераторы и генераторы

## Итераторы

__Итератор__ - некая конструкция, которая перебирает элементы некоего объекта.

Цикл for - это не что иное, как итератор.

Чтобы проитерировать некий объект (перебрать его элементы) у объекта (класса) должен быть метод __iter__, который возвращает нам итератор.

Для того, чтобы объект был итератором, у него должен быть метод __next__, который возвращает нам следующий элемент.

Итератор определяется функцией iter():

In [6]:
lst1 = [1, 2, 3, 4, 5]
book = {'Фредерик Бегбедер' : 'Любовь живет три года',
        'Джон Толкиен' : 'Властелин колец',
        'Кен Кизи' : 'Пролетая над гнездом кукушки'}
tpl = (0, -1, -2, -3)
str1 = 'Hello'

for el in lst1:      # перебираем элементы с помощью for
    print(el)
print()
for el in tpl:       # перебираем элементы с помощью for
    print(el)

1
2
3
4
5

0
-1
-2
-3


In [18]:
iter_lst1 = iter(lst1)
while True:                         #создаем бесконечный цикл
    try:                            # используем try-except на вылавливание ошибки "конец итератора"
        print(next(iter_lst1))
    except StopIteration:           # когда ошибка вылавливается - цикл останавливается
        break
        
print()
        
iter_book = iter(book)

while True:
    try:
        print(next(iter_book))
    except StopIteration:
        break

1
2
3
4
5

Фредерик Бегбедер
Джон Толкиен
Кен Кизи


Чтобы создать новый итератор, который подчиняется нашим правилам, мы можем создать новый класс:

In [51]:
import random
class RandomNum:
    def __init__(self, total, stop, start=0): # инициализируем итератор; по умолчанию необходимо передать только кол-во итераций
        self.total = total                # и правую границу заданного числового отрезка, из которого мы хотим выбирать число
        self.i = 0                          # счетчик
        self.start = start
        self.stop = stop
        
    def __next__(self):                    # инициализируем функцию next(); __next__(self) = next(self)
        if self.total > self.i:
            self.i += 1
            return random.randint(self.start,self.stop)
        else:
            raise StopIteration('Max amount of elements')
            
x = RandomNum(5,6)

In [52]:
print(next(x))
print(next(x))
print(next(x))
print(next(x))
print(next(x))
print(next(x))

3
3
2
2
6


StopIteration: Max amount of elements

Однако этого недостаточно для того, чтобы передать наш итератор в цикл for, как например мы передаем range():

In [46]:
for i in RandomNum(10, 10):
    print(i)

TypeError: 'RandomNum' object is not iterable

Появляется ошибка, что объект не итерируем.

Для того, чтобы это исправить, нам надо снова влезть в код нашего класса и добавить туда функцию iter():

In [47]:
import random
class RandomNumIterable:                   # для наглядности создадим новый класс
    def __init__(self, total, stop, start=0): # инициализируем итератор; по умолчанию необходимо передать только кол-во итераций
        self.total = total                # и правую границу заданного числового отрезка, из которого мы хотим выбирать число
        self.i = 0                          # счетчик
        self.start = start
        self.stop = stop
        
    def __next__(self):                    # инициализируем функцию next(); __next__(self) = next(self)
        if self.total > self.i:
            self.i += 1
            return random.randint(self.start,self.stop)
        else:
            raise StopIteration
            
    def __iter__(self):         # добавляем функцию __iter__, которая будет возвращать self (не что иное, как следующий)
        return(self)                                                                 # элемент итератора
            

In [53]:
for i in RandomNumIterable(10, 10):
    print(i)

3
4
2
5
6
4
1
9
9
2


## Генераторы

__Генератор__ - некая конструкция, которая генерирует некий итерируемый объект.

В отличие от итератора, генератор - это не класс, а функция, и создавать его значительно проще.

Важной особенностью такой функции является использование __yield__ вместо __return__: таким образом после возврата 1 значения функция не прекратит работу.

При этом мы не просто несколько раз исполняем функцию - мы возвращаем результат наружу и при этом запоминаем текущее состояние функции (например, мы не начинаем цикл __for__ заново.

Однако, если в теле функции есть еще какие-то действия после __yield__, и другой команды на вывод значения нет - код не исполнится

In [81]:
def random_num(k, stop, start = 0):
    for i in range(k):
        yield random.randint(start, stop)
    return 'Stop It Now!'                  #объяснено ниже по тексту

In [68]:
print(random_num(5,3))  # при этом если попробовать распечатать элементы - на экран выведется лишь сам объект

<generator object random_num at 0x000001199A80A7B0>


In [77]:
for el in random_num(5,9):
    print(el**2)

36
25
4
9
0


При этом если передать в функцию __return__ с неким сообщением - мы сможем вызвать ошибку StopIteration с этим сообщением:

In [80]:
gen = random_num(3,5)
print(next(gen))
print(next(gen))
print(next(gen))
print(next(gen))

0
2
3


StopIteration: Stop It Now!

Пример класса, построенного на генераторе:

In [89]:
def mul2(x):
    return x % 2 == 0

def mul3(x):
    return x % 3 == 0

def mul5(x):
    return x % 5 == 0



class multifilter:
    def judge_half(pos, neg):
        return pos >= neg

    def judge_any(pos, neg):
        return pos >= 1

    def judge_all(pos, neg):
        return neg == 0

    def __init__(self, iterable, *funcs, judge=judge_any):
        self.iterable = iterable
        self.funcs = funcs
        self.judge = judge
        

    def __iter__(self):
        for el in self.iterable:
            pos, neg = 0, 0
            pos = sum([f(el) for f in self.funcs])
            neg = len(self.funcs) - pos
            if self.judge(pos, neg):
                yield el
            else:
                continue






In [98]:
lst = list(range(5,125)) 
for i in multifilter(lst,mul2, mul3, mul5, judge=multifilter.judge_all):
    print(i)

30
60
90
120


In [99]:
def primes(k):
    for i in range(2, k+1):
        k = 0
        for m in range(1, i+1):
            if i % m == 0:
                k += 1
        if k == 2:
            yield i

In [114]:
def primes():
    i = 2
    counter = 1
    while True:
        k = 0
        for m in range(1, i+1):
            if i % m == 0:
                k += 1
                if k > 3:
                    break
        if k == 2:
          #  yield i
            counter += 1
        if i % 5000 == 0:
            print(f'В диапазоне {i - 5000} - {i} {counter} простых чисел')
            counter = 0
        i += 1  
        





for i in primes():
    print()

В диапазоне 0 - 5000 670 простых чисел
В диапазоне 5000 - 10000 560 простых чисел
В диапазоне 10000 - 15000 525 простых чисел
В диапазоне 15000 - 20000 508 простых чисел
В диапазоне 20000 - 25000 500 простых чисел
В диапазоне 25000 - 30000 483 простых чисел
В диапазоне 30000 - 35000 487 простых чисел
В диапазоне 35000 - 40000 471 простых чисел
В диапазоне 40000 - 45000 472 простых чисел
В диапазоне 45000 - 50000 458 простых чисел
В диапазоне 50000 - 55000 457 простых чисел
В диапазоне 55000 - 60000 467 простых чисел
В диапазоне 60000 - 65000 436 простых чисел
В диапазоне 65000 - 70000 442 простых чисел
В диапазоне 70000 - 75000 458 простых чисел
В диапазоне 75000 - 80000 444 простых чисел
В диапазоне 80000 - 85000 440 простых чисел
В диапазоне 85000 - 90000 436 простых чисел
В диапазоне 90000 - 95000 444 простых чисел
В диапазоне 95000 - 100000 435 простых чисел
В диапазоне 100000 - 105000 432 простых чисел
В диапазоне 105000 - 110000 429 простых чисел
В диапазоне 110000 - 115000 418 п

KeyboardInterrupt: 